In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import mysql.connector
from datetime import datetime

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/83.0.478.37",
    "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36"
]
def create_driver_with_user_agent(user_agent):
    options = Options()
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f'user-agent={user_agent}')
    return webdriver.Chrome(options=options)

def go_to_post_page(post_url):
    driver = create_driver_with_user_agent(random.choice(user_agents))
    driver.get(post_url)
    time.sleep(3)
    try:
        cookies = "/html/body/div[4]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        cookies2 = "/html/body/div[3]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        try:
            cookies_button = driver.find_element(By.XPATH, cookies)
        except:
            cookies_button = driver.find_element(By.XPATH, cookies2)
        cookies_button.click()
        time.sleep(2)
    except:
        delete_unavailable(post_url)
        driver.close()
        return None
    return driver

def connect_to_db():
    db_params = {
        'host': '192.168.100.200',
        'user': 'ecorradini',
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    return db_connection,db_cursor

def fetch_comments():
    db_connection,db_cursor = connect_to_db()
    query = """
        SELECT url, post
        FROM comment c
        WHERE NOT EXISTS (
            SELECT 1
            FROM data d
            WHERE d.url = c.url
        );
    """
    db_cursor.execute(query)
    return db_cursor.fetchall()

def insert_data_row(url, parent, user, caption, image_video_url, c_time, likes):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    if caption is not None:
        c = caption.replace('"','\"').replace("'","\'")
    else:
        c = caption
    insert_query = "INSERT IGNORE INTO data (url, parent, user, caption, image_video_url, time, likes) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    if c_time is not None:
        timestamp = datetime.strptime(c_time, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp()
        formatted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    else:
        formatted_time = c_time
    data = (url,parent,user,c,image_video_url,formatted_time,likes)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()
    
def insert_posts_into_data():
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    insert_query = """
        INSERT IGNORE INTO data (url, parent, user, caption, image_video_url, time, likes)
        SELECT
            p.post_url AS url,
            NULL AS parent,
            p.user_url AS user,
            p.caption as caption,
            p.image_url AS image_video_url,
            pd.time AS time,
            pd.likes as likes
        FROM post p
        JOIN post_data pd ON p.post_url = pd.url
    """
    db_cursor.execute(insert_query)
    db_connection.commit()
    db_cursor.close()
    db_connection.close()
    
def insert_comment(comment_url, post_url, user, image, caption, c_time):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    if caption is not None:
        c = caption.replace('"','\"').replace("'","\'")
    else:
        c = caption
    insert_query = "INSERT INTO comment (url, post, user, time, caption, image) VALUES (%s, %s, %s, %s, %s, %s)"
    if c_time is not None:
        timestamp = datetime.strptime(c_time, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp()
        formatted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    else:
        formatted_time = c_time
    data = (comment_url,post_url,user,formatted_time,c,image)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()
    
def delete_unavailable(comment_url):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    delete_query1 = f"DELETE FROM images WHERE post_url='{comment_url}'"
    db_cursor.execute(delete_query1)
    delete_query2 = f"DELETE FROM comment WHERE url='{comment_url}'"
    db_cursor.execute(delete_query2)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()

In [2]:
base_path = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]"
post_path = base_path + "/div[1]/div/div[2]/div/div/div[2]"
comments_path = base_path + "/div[position() > 1 and position() < last() - 2]"

def get_video_url(driver):
    video_path = post_path + "/div/div[3]/div/div[2]/div/div/div/div/div/div/div/div/div/div/video"
    try:
        video = driver.find_element(By.XPATH,video_path)
        video_url = video.get_attribute('src')
        return video_url
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        video_path2 = post_path2 + "/div/div[3]/div/div[2]/div/div/div/div/div/div/div/div/div/div/video"
        try:
            video2 = driver.find_element(By.XPATH,video_path2)
            video_url2 = video2.get_attribute('src')
            return video_url2
        except:
            return None

def get_image_url(driver):
    image_path = post_path + "/div[2]/div[3]/div/div[2]/div/div/picture/img"
    try:
        picture = driver.find_element(By.XPATH,image_path)
        image_url = picture.get_attribute('src')
        return image_url
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        image_path2 = post_path2 + "/div[2]/div[3]/div/div[2]/div/div/picture/img"
        try:
            picture2 = driver.find_element(By.XPATH,image_path2)
            image_url2 = picture2.get_attribute('src')
            return image_url2
        except:
            return None

def get_time(driver):
    time_path = post_path + "/div[2]/div/div[2]/span/a/time"
    try:
        time_p = driver.find_element(By.XPATH,time_path)
        post_time = time_p.get_attribute("datetime")
        return post_time
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        time_path2 = post_path2 + "/div[2]/div/div[2]/span/a/time"
        try:
            time_p2 = driver.find_element(By.XPATH,time_path2)
            post_time2 = time_p2.get_attribute("datetime")
            return post_time2
        except:
            return None

def get_user(driver):
    user_path = post_path + "/div[2]/div/div[1]/span/div/div/a"
    try:
        user = driver.find_element(By.XPATH,user_path)
        user_url = user.get_attribute("href")
        return user_url
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        user_path2 = post_path2 + "/div[2]/div/div[1]/span/div/div/a"
        try:
            user2 = driver.find_element(By.XPATH,user_path2)
            user_url2 = user2.get_attribute("href")
            return user_url2
        except:
            return None
    
def get_caption(driver):
    caption_path = post_path + "/div[3]/div/div[1]/span"
    try:
        caption = driver.find_element(By.XPATH,caption_path).text
        return caption
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        caption_path2 = post_path2 + "/div[3]/div/div[1]/span"
        try:
            caption2 = driver.find_element(By.XPATH,caption_path2).text
            return caption2
        except:
            return None

def get_likes(driver):
    likes_path = post_path + "/div[4]/div/a/span/span/div/div"
    try:
        return int(driver.find_element(By.XPATH,likes_path).text.replace(",", ""))
    except:
        post_path2 = base_path + "/div[1]/div/div[3]/div/div/div[2]"
        likes_path2 = post_path2 + "/div[4]/div/a/span/span/div/div"
        try:
            return int(driver.find_element(By.XPATH,likes_path2).text.replace(",", ""))
        except:
            return None
    
def parse_comments(driver):
    comment_base_path = "./div/div[1]/div/div[2]"
    comment_url_path = comment_base_path + "/div[2]/div/div[2]/span/a"
    user_path = comment_base_path + "/div[2]/div/div[1]/span/div/div/a"
    time_path = comment_base_path + "/div[2]/div/div[2]/span/a/time"
    contents_path = comment_base_path + "/div[3]/div"
    contents_path2 = comment_base_path + "/div[4]/div"
    comment_url = driver.find_element(By.XPATH,comment_url_path).get_attribute('href')
    user = driver.find_element(By.XPATH,user_path)
    user_url = user.get_attribute('href')
    time = driver.find_element(By.XPATH,time_path)
    comment_time = time.get_attribute("datetime")
    contents = driver.find_elements(By.XPATH,contents_path)
    if len(contents) > 0:
        caption,image = parse_comment_contents(contents)
        if caption == "":
            contents2 = driver.find_elements(By.XPATH,contents_path2)
            caption,image = parse_comment_contents(contents2)
    return comment_url,user_url,comment_time,caption,image
    
def parse_comment_contents(comment_contents):
    #caption_path = "./div[1]"
    caption = comment_contents[0].text
    image_path = "./div/div/picture/img"
    try:
        image = comment_contents[1].find_element(By.XPATH,image_path).get_attribute('src')
    except:
        image = ""
    return caption,image

In [4]:
while True:
    comments = [(x[0],x[1]) for x in fetch_comments()]
    if len(comments) > 0:
        comment = comments[0][0]
        post = comments[0][1]
        driver = go_to_post_page(comment)
        if driver is not None:
            user = get_user(driver)
            caption = get_caption(driver)
            image_video = get_image_url(driver)
            if image_video is None:
                image_video = get_video_url(driver)
            p_time = get_time(driver)
            likes = get_likes(driver)
            insert_data_row(comment, post, user, caption, image_video, p_time, likes)
            comments = driver.find_elements(By.XPATH,comments_path)
            for comment in comments:
                try:
                    comment_url,comment_user,comment_time,comment_caption,comment_image = parse_comments(comment)
                    insert_comment(comment_url,post,comment_user, comment_image, comment_caption,comment_time)
                except:
                    continue
            driver.close()
        insert_posts_into_data()

KeyboardInterrupt: 

In [ ]:
comments = [(x[0],x[1]) for x in fetch_comments()]
driver = go_to_post_page(comments[0][0])

In [ ]:
user = get_user(driver)
caption = get_caption(driver)
image_video = get_image_url(driver)
if image_video is None:
    image_video = get_video_url(driver)
p_time = get_time(driver)
likes = get_likes(driver)

print(user)
print(caption)
print(image_video)
print(p_time)
print(likes)

In [37]:
comments = driver.find_elements(By.XPATH,comments_path)
parse_comments(comments[0])

('https://www.threads.net/@vikkotaruc/post/C02hupxyV0a',
 'https://www.threads.net/@vikkotaruc',
 '2023-12-14T23:31:41.000Z',
 'And I respect you the same, man!',
 '')